In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))
sys.path.append(os.path.join(os.getcwd(), "iclr19-graph2graph/props"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, load_checkpoint, initialize_models
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.ot_utils import encode_target
from mol_opt.ot_utils import FGW 
from mol_opt.ot_utils import compute_barycenter

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty, RecPenalty
from molgen.metrics.mol_metrics import MolMetrics

from rdkit.Chem import MolFromSmiles
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import grad
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from molgen.metrics.Penalty import Penalty as PenaltyNew

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

/Users/octavdragoi/gitrepos/tum-thesis


In [2]:
model_type = "transformer-ae"
sys.argv = ["", "-model_type", model_type, "-one_batch_train"]
args = get_args()

molopt = MolOpt(args)
molopt_decoder = MolOptDecoder(args)

In [4]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, True)
datatype = "train_split"
train_data_loader = get_loader("molgen/data/chembl50", datatype, 50, same_number_atoms = True)

for i in train_data_loader:
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
    X = MolGraph(i)
    Y = X
    break

In [5]:
x_embedding = molopt.encode(X)

In [6]:
from mol_opt.transformer import make_model
from torch.autograd import Variable
model = make_model(args)
# model.forward(x_embedding, x_embedding).shape

In [7]:
model(x_embedding, x_embedding, None, None)

tensor([[ 0.8087,  0.8461,  0.4251,  ..., -0.0404,  0.7888,  0.3034],
        [-0.3296, -0.0846, -0.3621,  ..., -0.2109,  0.4884,  1.1222],
        [-0.0584, -0.7211,  0.8396,  ..., -0.4621, -0.6480,  0.2183],
        ...,
        [-0.0419,  1.0639,  0.0635,  ..., -0.2797, -0.1361,  1.2858],
        [-0.4359, -1.4716,  0.0486,  ..., -0.0195, -0.8645,  0.6632],
        [-0.2142,  0.6243, -0.8473,  ..., -0.5514, -0.4905,  0.7442]],
       grad_fn=<AddBackward0>)

In [21]:
i = 5
x_embedding[i].shape
ys = torch.zeros((1,args.pc_hidden))

for i in range(X.scope[i][1]):
    print (i, ys.shape)
    out = model(x_embedding[i], Variable(ys), None, None)
    ys = torch.cat([ys, out[-1].unsqueeze(0)])
#     ys = torch.cat([ys, 
#                     torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

IndexError: tuple index out of range

In [23]:
x_embedding

AttributeError: 'tuple' object has no attribute 'shape'

In [10]:
out.shape

torch.Size([23, 150])

In [11]:
x_embedding[i].shape

torch.Size([150])

In [12]:
x_embedding.shape[]

SyntaxError: invalid syntax (<ipython-input-12-e4dd6f5b256a>, line 1)

In [29]:
x_encoding, x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize_gumbel(*yhat_logits, tau = pen.tau)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

pen.mirror = ""

pen.tau = 0.05
fgw_loss_item = fgw_loss(*pred_pack, tau = 1)
print (fgw_loss_item.item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

0 16
0 torch.Size([1, 150]) torch.Size([150])
1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
torch.Size([1, 16, 150])
16 29
0 torch.Size([1, 150]) torch.Size([150])
1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torc

2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
17 torch.Size([18, 150]) torch.Size([150])
18 torch.Size([19, 150]) torch.Size([150])
torch.Size([1, 18, 150])
209 16
0 torch.Size([1, 150]) torch.Size([150])
1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torc

torch.Size([1, 25, 150])
377 21
0 torch.Size([1, 150]) torch.Size([150])
1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
17 torch.Size([18, 150]) torch.Size([150])
18 torch.Size([19, 150]) torch.Size([150])
19 torch.Size([20, 150]) torch.Size([150])
20 torch.Size([21, 150]) torch.Size([150])
21 torch.Size([22, 150]) torch.Size([150])
torch.Size([1, 21, 150])
398 14
0 torch.S

7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
17 torch.Size([18, 150]) torch.Size([150])
18 torch.Size([19, 150]) torch.Size([150])
19 torch.Size([20, 150]) torch.Size([150])
20 torch.Size([21, 150]) torch.Size([150])
21 torch.Size([22, 150]) torch.Size([150])
22 torch.Size([23, 150]) torch.Size([150])
23 torch.Size([24, 150]) torch.Size([150])
24 torch.Size([25, 150]) torch.Size([150])
25 torch.Size([26, 150]) torch.Size([150])
26 torch.Size([27, 150]) torch.Size([150])
27 torch.Size([28, 150]) torch.Size([150])
28 torch.Size([29, 150]) torch.Size([150])
torch.Size([1, 28, 150])
580 13
0 torch.Size([1, 150]) torc

3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
17 torch.Size([18, 150]) torch.Size([150])
18 torch.Size([19, 150]) torch.Size([150])
19 torch.Size([20, 150]) torch.Size([150])
20 torch.Size([21, 150]) torch.Size([150])
21 torch.Size([22, 150]) torch.Size([150])
22 torch.Size([23, 150]) torch.Size([150])
23 torch.Size([24, 150]) torch.Size([150])
24 torch.Size([25, 150]) torch.Size([150])
25 torch.Size([26, 150]) torch.Size([150])
26 torch.Size([27, 150])

29 torch.Size([30, 150]) torch.Size([150])
30 torch.Size([31, 150]) torch.Size([150])
31 torch.Size([32, 150]) torch.Size([150])
32 torch.Size([33, 150]) torch.Size([150])
33 torch.Size([34, 150]) torch.Size([150])
34 torch.Size([35, 150]) torch.Size([150])
35 torch.Size([36, 150]) torch.Size([150])
36 torch.Size([37, 150]) torch.Size([150])
37 torch.Size([38, 150]) torch.Size([150])
38 torch.Size([39, 150]) torch.Size([150])
39 torch.Size([40, 150]) torch.Size([150])
40 torch.Size([41, 150]) torch.Size([150])
41 torch.Size([42, 150]) torch.Size([150])
42 torch.Size([43, 150]) torch.Size([150])
43 torch.Size([44, 150]) torch.Size([150])
torch.Size([1, 43, 150])
940 20
0 torch.Size([1, 150]) torch.Size([150])
1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([1

1 torch.Size([2, 150]) torch.Size([150])
2 torch.Size([3, 150]) torch.Size([150])
3 torch.Size([4, 150]) torch.Size([150])
4 torch.Size([5, 150]) torch.Size([150])
5 torch.Size([6, 150]) torch.Size([150])
6 torch.Size([7, 150]) torch.Size([150])
7 torch.Size([8, 150]) torch.Size([150])
8 torch.Size([9, 150]) torch.Size([150])
9 torch.Size([10, 150]) torch.Size([150])
10 torch.Size([11, 150]) torch.Size([150])
11 torch.Size([12, 150]) torch.Size([150])
12 torch.Size([13, 150]) torch.Size([150])
13 torch.Size([14, 150]) torch.Size([150])
14 torch.Size([15, 150]) torch.Size([150])
15 torch.Size([16, 150]) torch.Size([150])
16 torch.Size([17, 150]) torch.Size([150])
17 torch.Size([18, 150]) torch.Size([150])
18 torch.Size([19, 150]) torch.Size([150])
19 torch.Size([20, 150]) torch.Size([150])
20 torch.Size([21, 150]) torch.Size([150])
21 torch.Size([22, 150]) torch.Size([150])
22 torch.Size([23, 150]) torch.Size([150])
23 torch.Size([24, 150]) torch.Size([150])
24 torch.Size([25, 150]) tor

NameError: name 'pen' is not defined

In [27]:
2250/150

15.0

In [14]:
ys.shape

torch.Size([24, 150])

In [31]:
memory = model.encode(x_embedding)

In [40]:
model(memory, x_embedding)

NameError: name 'memory' is not defined

In [104]:
max_len = 5 
ys = torch.zeros(50, args.pc_hidden)
for i in range(max_len-1):
    out = model.decode(memory,
                       Variable(ys)) 
    ys = torch.cat((ys, out)).view(50,2,150)

RuntimeError: The size of tensor a (2) must match the size of tensor b (100) at non-singleton dimension 1

In [105]:
out.shape

torch.Size([50, 150])

In [106]:
ys.shape

torch.Size([50, 2, 150])

In [53]:
torch.from_numpy(np.random.randint(1, 20, size=(50, 10))).shape

torch.Size([50, 10])

In [34]:
x_encoding, Z = molopt.forward(X)

AttributeError: 'NoneType' object has no attribute 'mean'

In [70]:
molopt.transformer(Z[0], None).shape

torch.Size([1, 150])

In [52]:
Z.unsqueeze(1).shape

torch.Size([50, 1, 150])

In [73]:
Z.unsqueeze(1).repeat(1,5,1).shape

torch.Size([50, 5, 150])

In [117]:
molopt.transformer(x_embedding[1], None).shape

torch.Size([1, 150])

In [120]:
x_embedding[1].shape

torch.Size([150])